In [1]:
import sys 
sys.path.append("../../code")
from model.cnn_geometric_model import *

In [3]:
#Using their preset network to figure out dimensions
affine = CNNGeometric(output_dim=6, use_cuda=False)
tps = CNNGeometric(output_dim=18, use_cuda=False)

In [4]:
#Getting children
affine.children

<bound method Module.children of CNNGeometric(
  (FeatureExtraction): FeatureExtraction(
    (model): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1)

In [6]:
x = torch.empty((1,3,240,240))
x1 = affine.FeatureExtraction(x)
x2 = affine.FeatureCorrelation(x1,x1)

#FeatureRegression has a Conv and Linear part. 
x3 = affine.FeatureRegression.conv(x2)
x4 = affine.FeatureRegression.linear(x3.reshape(1,-1)) #Need to reshape x3 to make it fit into the linear.

print("Input shape:\n", x.shape)
print("After Feat.Extract,\n", x1.shape)
print("After Feat.Corr,\n", x2.shape)
print("After Feat Reg Conv,\n", x3.shape)
print("After Feat Reg Lin,\n", x4.shape)

Input shape:
 torch.Size([1, 3, 240, 240])
After Feat.Extract,
 torch.Size([1, 512, 15, 15])
After Feat.Corr,
 torch.Size([1, 225, 15, 15])
After Feat Reg Conv,
 torch.Size([1, 64, 5, 5])
After Feat Reg Lin,
 torch.Size([1, 6])


In [7]:
#Pedro's model
pedro_fe = nn.Sequential(
    nn.Conv2d(3,128,3,padding=1), #Better using little kernels if we want to detect small objects. Also use odd numbers 
    nn.ReLU(),
    nn.Conv2d(128,256,3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),

    nn.Conv2d(256,512,3,padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    nn.Conv2d(512,512,5,padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
    nn.Conv2d(512,512,5,padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            )

In [5]:
#Richie's model
x = torch.empty((1,3,240,240),device='cuda')
def bn2d(x): return nn.BatchNorm2d(x)
leaky = nn.LeakyReLU()
fe = nn.Sequential(nn.Conv2d(3,32, kernel_size = (5,5)),
                   bn2d(32),
                   leaky,
                   nn.Conv2d(32,64, kernel_size = (5,5)),
                   bn2d(64),
                   nn.MaxPool2d(kernel_size=2),
                   nn.ReLU(),
                   nn.Conv2d(64,128, kernel_size = (5,5)),
                   bn2d(128),
                   leaky,
                   nn.Conv2d(128,256, kernel_size = (5,5)),
                   nn.MaxPool2d(kernel_size=2),
                   nn.ReLU(),
                   nn.Conv2d(256,512, kernel_size = (5,5)),
                   bn2d(512),
                   leaky,
                   nn.Conv2d(512,512, kernel_size = (5,5)),
                   nn.MaxPool2d(kernel_size=3),
                   nn.ReLU())
fe.cuda()
print(fe)
print(fe(x).shape)

Sequential(
  (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): ReLU()
  (7): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): LeakyReLU(negative_slope=0.01)
  (10): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): ReLU()
  (13): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
  (14): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (15): LeakyReLU(negative_slope=0.01)
  (16): Conv2d(512, 512, kernel_size=

python -W ignore train.py --training-dataset rgb512_aug --image-size 512 --feature-extraction-cnn resnet101 --geometric-model affine --pretrained False --trained-model-dir ./trained_models/ --trained-model-fn baseline_resnet_affine --num-epochs 40 --lr 0.00678  --momentum 0.915



### `wormbrain_1`

##### Affine
```
python -W ignore train.py --training-dataset rgb512_aug --geometric-model affine --feature-extraction-cnn wormbrain_1 --trained-model-dir ./trained_models/test_arch --trained-model-fn new_std --num-epochs 35 --lr 0.000578 --image-size 512 --momentum 0.93
```

##### TPS
```
python -W ignore train.py --training-dataset rgb512_aug --geometric-model tps --feature-extraction-cnn wormbrain_1 --trained-model-dir ./trained_models/test_arch --trained-model-fn TPS_hyperparms --num-epochs 35 --lr 0.000878 --image-size 512 --momentum 0.93 --weight-decay 0.0009
```

In [11]:
from util.handle_files import *
from skimage import io
import numpy as np
                               
x = list_files('../datasets/rgb512_augmented/TrainVal/', 'jpg')
imgs = []             
for f in x :
    imgs.append(io.imread(f))
arr = np.asarray(imgs)

In [12]:
print("red",arr[...,0].mean()/255, arr[...,0].std()/255)
print("green", arr[...,1].mean()/255, arr[...,1].std()/255)

red 0.0742920482408156 0.01979817471748977
green 0.0754988783485235 0.03203817782416048


In [13]:
print("blue", arr[...,2].mean()/255, arr[...,2].std()/255)

blue 0.00703948247409379 0.015462965792484733


#### CHANGING THE FEATURE REGRESSOR

I've found the feature regressor to be quite (too?) simple, with no operations.
I decided to add a small MLP for feature regression prediction, see below

```
self.linear = nn.Linear(ch_out * kernel_sizes[-1] * kernel_sizes[-1], output_dim)
```

is changed into : 

```
self.Dropout = nn.Dropout(0.25)
self.linear = nn.Sequential(
                nn.Linear(ch_out * kernel_sizes[-1] * kernel_sizes[-1], 1024),
                nn.ReLU(),
                nn.BatchNorm1d(1024),
                self.Dropout,
                nn.Linear(1024,512),
                nn.ReLU(),
                nn.BatchNorm1d(512),
                self.Dropout,
                nn.Linear(512,256),
                nn.ReLU(),
                nn.BatchNorm1d(256),
                self.Dropout,
                nn.Linear(256,128),
                nn.ReLU(),
                nn.BatchNorm1d(128),
                self.Dropout,
                nn.Linear(128,64),
                nn.ReLU(),
                nn.BatchNorm1d(64),
                self.Dropout,
                nn.Linear(64, output_dim))
```

This helped me regularize training.

Training parameters used for this new Feat. Reg. and architecture "wormbrain_2" (to use ELU instead of LeakyReLU)

Using a smaller training set to be quicker (3000 images vs 5720).

#### AFFINE
```
python -W ignore train.py --training-dataset smallset --geometric-model affine --feature-extraction-cnn wormbrain_2 --trained-model-dir ./trained_models/test_arch --trained-model-fn new_std --num-epochs 35 --lr 0.001234 --image-size 512 --momentum 0.91
```
```
python -W ignore train.py --training-dataset rgb512_aug --geometric-model affine --feature-extraction-cnn wormbrain_1 --trained-model-dir ./trained_models/test_arch/wormbrain_1/ --trained-model-fn new_featreg_relu --num-epochs 40 --lr 0.02667 --image-size 512 --momentum 0.9
```
```
python -W ignore train.py --training-dataset rgb512_aug --geometric-model affine --feature-extraction-cnn wormbrain_1 --trained-model-dir ./trained_models/test_arch/wormbrain_2/ --trained-model-fn new_featreg_relu --num-epochs 40 --lr 0.01667 --image-size 512 --momentum 0.95
```
```
python -W ignore train.py --training-dataset smallset --geometric-model affine --feature-extraction-cnn wormbrain_1 --trained-model-dir ./trained_models/test_arch/smallset/ --trained-model-fn t1_new_featreg_relu --num-epochs 40 --lr 0.001667 --image-size 512 --momentum 0.92
```
```
python -W ignore train.py --training-dataset smallset --geometric-model tps --feature-extraction-cnn wormbrain_1 --trained-model-dir ./trained_models/test_arch/smallset/ --trained-model-fn wormbrain_1 --num-epochs 40 --lr 0.003667 --image-size 512 --momentum 0.915 
```

### Trying to train the new regressor + Richie's architecture using the full dataset : 
These are the "best" overall I got.

I haven't logged the results for the other as they were either unstable 
(loss oscillate between epochs) or didn't converge to a low enough value.

#### **Affine** : 

``` 
python -W ignore train.py --training-dataset rgb512_aug --geometric-model affine --feature-extraction-cnn wormbrain_1 --trained-model-dir ./trained_models/test_arch/fullset/ --trained-model-fn wormbrain_1_newFR_dropout --num-epochs 40 --lr 0.0023667 --image-size 512 --momentum 0.92 
```
With Dropout set to 0.25, using wormbrain_1 (LeakyReLU instead of ELU)
    
Final epoch (40) : Train loss = 0.280 , Validation loss = 0.252** 

BEST epoch (35) : Train loss = 0.262, Validation loss = 0.197**

-------------------------
```
python -W ignore train.py --training-dataset rgb512_aug --geometric-model affine --feature-extraction-cnn wormbrain_1 --feature-regression wormbrain --fr-dropout 0.2 --trained-model-dir ./trained_models/test_arch/fullset/ --trained-model-fn AffineWB_dropout02 --num-epochs 50 --lr 0.004667 --image-size 512 --momentum 0.92 --lr_max_iter 10000
```

With Dropout set to 0.2, wb1

**Final epoch (50) : Train loss = 0.293 , Validation loss = 0.281** 

**BEST epoch (36) : Train loss = 0.269, Validation loss = 0.204**

-----------------------
#### **Thin Plate Spline**
```
python -W ignore train.py --training-dataset rgb512_aug --geometric-model tps --feature-extraction-cnn wormbrain_2 --trained-model-dir ./trained_models/test_arch/fullset/ --trained-model-fn TPS_wormbrain_2_newFR_dropout015xd --num-epochs 50 --lr 0.003667 --image-size 512 --momentum 0.94
```
 
With dropout = 0.2, using wormbrain_2 (ELU instead of LeakyReLU)

    **Final result : Train loss = 0.411 , Validation loss = 0.397** 
    
    **BEST epoch (47) : Train loss = 0.406, Validation loss = 0.378**

------------------------

    
```
python -W ignore train.py --training-dataset rgb512_aug --geometric-model tps --feature-extraction-cnn wormbrain_2 --feature-regression wormbrain --fr-dropout 0.35 --trained-model-dir ./trained_models/test_arch/fullset/ --trained-model-fn tpsWB_dropout035 --num-epochs 50 --lr 0.005667 --image-size 512 --momentum 0.935 --lr_max_iter 7500
```

With dropout = 0.35, wb2
training set : Average loss: 0.0465
Validation set: Average loss: 0.0464




## Simpler Feature Regressor

Having tried to train the feature regressor with TPS many times with no success (doesn't go much below 0.379 validation loss), I've decided to reduce the complexity of the model, as well as adding a softmax activation in the early layer to normalize the features to (0,1), i.e. a probability distribution, and have the network act like a multinomial logistic regression.

Thus, the model is reduced from :
```
self.Dropout = nn.Dropout(0.25)
self.linear = nn.Sequential(
                nn.Linear(ch_out * kernel_sizes[-1] * kernel_sizes[-1], 1024),
                nn.ReLU(),
                nn.BatchNorm1d(1024),
                self.Dropout,
                nn.Linear(1024,512),
                nn.ReLU(),
                nn.BatchNorm1d(512),
                self.Dropout,
                nn.Linear(512,256),
                nn.ReLU(),
                nn.BatchNorm1d(256),
                self.Dropout,
                nn.Linear(256,128),
                nn.ReLU(),
                nn.BatchNorm1d(128),
                self.Dropout,
                nn.Linear(128,64),
                nn.ReLU(),
                nn.BatchNorm1d(64),
                self.Dropout,
                nn.Linear(64, output_dim))
```

to 

```
self.Dropout = nn.Dropout(p_dropout)
            self.linear = nn.Sequential(
                nn.Linear(ch_out * kernel_sizes[-1] * kernel_sizes[-1], 512),
                nn.BatchNorm1d(512), 
                nn.Softmax(dim=1),
                self.Dropout,
                
                nn.Linear(512,128),
                nn.ReLU(),
                nn.BatchNorm1d(128),
                self.Dropout,
                
                nn.Linear(128,output_dim))
```

This allowed the best average validation loss to reach around 0.0304 (versus 0.5 before), as well as converging earlier without re-increasing. (~20 epochs, versus 30-35 epochs before) 